<a href="https://colab.research.google.com/github/andykofman/Postura/blob/main/Postura_Live_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Get the project code**

In [1]:
# Clean workspace repo folder
!cd /content
!rm -rf Postura
!git clone https://github.com/andykofman/Postura.git

Cloning into 'Postura'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 208 (delta 77), reused 128 (delta 43), pack-reused 41 (from 1)
Receiving objects: 100% (208/208), 35.22 MiB | 37.26 MiB/s, done.
Resolving deltas: 100% (81/81), done.


**2) Change dir to Postura**

In [2]:
%cd /content/Postura

/content/Postura


**3) Remove Colab extras that can force protobuf>=5 (conflicts with MediaPipe)**

In [3]:
!pip uninstall -y ydf grpcio-status || true

Found existing installation: ydf 0.13.0
Uninstalling ydf-0.13.0:
  Successfully uninstalled ydf-0.13.0
Found existing installation: grpcio-status 1.71.2
Uninstalling grpcio-status-1.71.2:
  Successfully uninstalled grpcio-status-1.71.2


**4) Install pinned dependencies (deterministic)**

In [4]:
!python -V
!python -m pip install --upgrade pip
!pip install -r requirements.txt -c constraints.txt

Python 3.11.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 103.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 57.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 49.0 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 149.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 58.5 MB/s  0:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.2.10
    Uninstalling flatbuffers-25.2.10:
      Successfully unins

**5) Hard-restart runtime so imports use the final environment**

if already restarted, skip to **step 6**

In [ ]:
import os, sys
os.kill(os.getpid(), 9)

**6) (After restart) Re-enter the repo**

In [1]:
%cd /content/Postura

/content/Postura


**7) Sanity checks (idempotent)**

In [ ]:
!python scripts/colab_bootstrap.py --remove-colab-conflicts --run-mp-smoke

**8) Start the API server in the background**

In [2]:
!pkill -f uvicorn || true
!nohup uvicorn api.main:app --host 0.0.0.0 --port 8000 --workers 1 > uvicorn.log 2>&1 &
!sleep 2 && sed -n '1,60p' uvicorn.log

^C
INFO:     Started server process [964]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


**9) Open ngrok tunnel (authenticated)**

Enter: **url/ui**


In [5]:
%pip -q install pyngrok
from pyngrok import ngrok
ngrok.kill()  # ensure clean slate
ngrok.set_auth_token("31Ma6WurjHpwLKZYddslQ16M8v5_7QcAgcGLyeZty1bZ7eupf")
tunnel = ngrok.connect(8000, "http", bind_tls=True)
public_url = tunnel.public_url
public_url, f"{public_url}/ui"

('https://f17c85bac0af.ngrok-free.app',
 'https://f17c85bac0af.ngrok-free.app/ui')

**12) Stop the server and tunnel when done**

In [7]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
!pkill -f uvicorn || true

In [9]:
%pwd

'/content/Postura'

In [8]:
# If you ran the ngrok cell, `public_url` is defined. Otherwise fallback to proxy/localhost.
import os, time, json, requests
try:
    BASE = public_url
except NameError:
    try:
        from google.colab import output
        BASE = output.eval_js("google.colab.kernel.proxyPort(8000)")
    except Exception:
        BASE = "http://127.0.0.1:8000"
print("BASE:", BASE)

def submit_and_wait(path):
    with open(path,'rb') as f:
        r = requests.post(f"{BASE}/analyze", files={"file": (os.path.basename(path), f, "video/mp4")})
    r.raise_for_status()
    job = r.json()["video_id"]
    print("job:", job)
    # poll until done
    while True:
        s = requests.get(f"{BASE}/status/{job}").json()
        if s.get("status") in {"done","error"}:
            print("status:", s.get("status"))
            break
        time.sleep(1)
    res = requests.get(f"{BASE}/result/{job}").json()
    print(json.dumps(res.get("summary", res), indent=2))
    print("Results page:", f"{BASE}/ui/result.html?video_id={job}")
    print("PDF:", f"{BASE}/report/{job}.pdf")
    return job, res

# Try a squat demo
submit_and_wait("/content/Postura/demo/squats.mp4")

BASE: https://f17c85bac0af.ngrok-free.app


FileNotFoundError: [Errno 2] No such file or directory: 'demo/squats.mp4'